In [2]:
import nltk
nltk.download('semcor')


[nltk_data] Downloading package semcor to C:\Users\L5
[nltk_data]     15IMH6-58IX\AppData\Roaming\nltk_data...
[nltk_data]   Package semcor is already up-to-date!


True

In [3]:
from nltk.corpus import semcor
import nltk

tagged_sentences = semcor.tagged_sents(tag='both')

word_lemma_pairs = [[(tree.leaves(), tree.label() if isinstance(tree.label(), nltk.corpus.reader.wordnet.Lemma) else None) 
                      for tree in sentence] 
                     for sentence in tagged_sentences]

print(word_lemma_pairs[:3])  # print the first 3 sentences to check


[[(['The'], None), (['Fulton', 'County', 'Grand', 'Jury'], Lemma('group.n.01.group')), (['said'], Lemma('state.v.01.say')), (['Friday'], Lemma('friday.n.01.Friday')), (['an'], None), (['investigation'], Lemma('probe.n.01.investigation')), (['of'], None), (['Atlanta'], Lemma('atlanta.n.01.Atlanta')), (["'s"], None), (['recent'], Lemma('late.s.03.recent')), (['primary', 'election'], Lemma('primary.n.01.primary_election')), (['produced'], Lemma('produce.v.04.produce')), (['``'], None), (['no'], None), (['evidence'], Lemma('evidence.n.01.evidence')), (["''"], None), (['that'], None), (['any'], None), (['irregularities'], Lemma('abnormality.n.04.irregularity')), (['took', 'place'], Lemma('happen.v.01.take_place')), (['.'], None)], [(['The'], None), (['jury'], Lemma('jury.n.01.jury')), (['further'], Lemma('far.r.02.far')), (['said'], Lemma('state.v.01.say')), (['in'], None), (['term'], Lemma('term.n.02.term')), (['end'], Lemma('end.n.02.end')), (['presentments'], Lemma('presentment.n.01.pres

In [4]:
def create_context_window(word_lemma_pairs, window_size=2):
    """
    Create context windows for each word in each sentence.

    Each data point will consist of a word, its sense, and the words 
    in its context window. If a word is near the beginning or end of 
    a sentence, the context window will be padded with None.

    Args:
        word_lemma_pairs: List of sentences, where each sentence is a
            list of (word, lemma) pairs.
        window_size: The number of words to include in the context
            window on each side of the target word.

    Returns:
        A list of tuples, where each tuple is (context, word, lemma).
        context is a list of words, word is a string, and lemma is a
        Synset object or None.
    """
    data = []
    for sentence in word_lemma_pairs:
        # Pad the sentence with None for context windows at the start and end
        padded_sentence = [(None, None)]*window_size + sentence + [(None, None)]*window_size
        for i in range(window_size, len(padded_sentence) - window_size):
            context = [padded_sentence[i+j][0] for j in range(-window_size, window_size+1) if j != 0]
            word, lemma = padded_sentence[i]
            data.append((context, word, lemma))
    return data

data = create_context_window(word_lemma_pairs)
data[:5]  # show the first 5 data points


[([None, None, ['Fulton', 'County', 'Grand', 'Jury'], ['said']],
  ['The'],
  None),
 ([None, ['The'], ['said'], ['Friday']],
  ['Fulton', 'County', 'Grand', 'Jury'],
  Lemma('group.n.01.group')),
 ([['The'], ['Fulton', 'County', 'Grand', 'Jury'], ['Friday'], ['an']],
  ['said'],
  Lemma('state.v.01.say')),
 ([['Fulton', 'County', 'Grand', 'Jury'], ['said'], ['an'], ['investigation']],
  ['Friday'],
  Lemma('friday.n.01.Friday')),
 ([['said'], ['Friday'], ['investigation'], ['of']], ['an'], None)]

In [5]:
from sklearn.model_selection import train_test_split

# Assuming "data" is your list of (context, word, lemma) tuples
X = [x[0] for x in data]  # input data (context words)
y = [x[2] for x in data]  # target data (word senses)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer object
vectorizer = CountVectorizer()

# Fit the vectorizer to the training data and transform the training data
# Transform the testing data
X_train_vect = vectorizer.fit_transform([' '.join(word[0] if isinstance(word, list) else '' for word in context) for context in X_train])
X_test_vect = vectorizer.transform([' '.join(word[0] if isinstance(word, list) else '' for word in context) for context in X_test])


In [7]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='log')
clf.fit(X_train_vect, y_train_str)

y_pred = clf.predict(X_test_vect)

accuracy = accuracy_score(y_test_str, y_pred)

print("Accuracy:", accuracy)


NameError: name 'y_train_str' is not defined

In [9]:
# Determine the number of samples to take
sample_size = int(X_train_vect.shape[0] * 0.010)

# Slice the training data
X_train_sample = X_train_vect[:sample_size]
y_train_sample = y_train[:sample_size]

# Convert lemmas to strings
y_train_sample_str = [str(lemma) for lemma in y_train_sample]

# Now you can try fitting the Naive Bayes model again
from sklearn.naive_bayes import BernoulliNB

clf = BernoulliNB()
clf.fit(X_train_sample, y_train_sample_str)


BernoulliNB()

##### stops here